### Task 1: Describe a machine learning problem that you would like to solve using Logistic Regression. Clearly state why Logistic regression is the best choice for solving this problem.

    I would like to solve better smoke detection using logistic regression. Currently there are two major competing types of smoke detectors, photoelectric smoke detectors that rely on shining a source of light through the air onto on a photosensor and ionization smoke detectors that ionize the air and measure the electrical resistance of the air that fluctuates based on particulates (such as smoke). Both are highly effective simple devices, however, there is still room for improvement as false alarms and late detections are still open issues with these devices. 

    More complex devices may solve these issues by intergrating multiple sensors to detect fires with higher accuracy and not accidentally trigger with things like cooking smoke or water vapor. These multi-sensor devices are perfect for logisitc regression based classifiers as the models can be easily trained, verified, and most importantly the trained model can run on an intergrated circuit such as the ones used in smoke detectors.


### I Will use the following dataset to build a classifier to detect fire based on multiple sensors
https://www.kaggle.com/datasets/deepcontractor/smoke-detection-dataset


### Exploratory Data Analysis (EDA)





In [1]:
pip install numpy matplotlib pandas

  Using cached numpy-1.23.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached matplotlib-3.5.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.3 MB)
     |████████████████████████████████| 11.7 MB 1.5 MB/s eta 0:00:01
  Using cached fonttools-4.37.1-py3-none-any.whl (957 kB)
  Using cached kiwisolver-1.4.4-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached Pillow-9.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
Note: you may need to restart the kernel to use updated packages.
